In [1]:
# imports

import krakenex as k
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import time
import datetime
from pathlib import Path
import sys

In [2]:
#funciones auxiliares

from datetime import datetime, timedelta
# Convert a unix time u to a datetime object d
def unixtoDate(u): 
    return datetime.fromtimestamp(u) + timedelta(hours=1)

def strToNum(s):
    try:
        return int(s)
    except ValueError:
        return float(s)
    
def reverseCumSum(serie):
    serieAcum = serie
    volAcum = float(0.0)
    for i in range(len(serie),0,-1):
        volAcum += float(serie.values[i-1])
        serieAcum.values[i-1]= volAcum
    return serie

def PassTime(d1, d2):
    #d1 = datetime.strptime(d1, "%Y-%m-%d %H:%M:%S")
    #d2 = datetime.strptime(d2, "%Y-%m-%d %H:%M:%S")
    return abs((d2 - d1).seconds)

In [3]:
#set SQL BBDD
from sqlalchemy import create_engine, MetaData, Table, Column, DateTime, Float, String,Integer

dbInstance = 'sqlite:///krakenTeader.db'
dbBalanceHistoryTable = 'BalanceHistory'
dbTradesHistoryTable = 'TradesHistory'
engine = create_engine(dbInstance)

if not engine.dialect.has_table(engine, dbBalanceHistoryTable):  # If table don't exist, Create.
    metadata = MetaData(engine)
    
    # Create a table with the appropriate Columns
    Table(dbBalanceHistoryTable, metadata,
          Column('Time', DateTime, primary_key=True, nullable=False), 
          Column('close', Float), 
          Column('ask', Float), 
          Column('bid', Float),
          Column('balanceRatio', Float),
          Column('volbuy', Float), 
          Column('volsell', Float),
          Column('unbalance', Float))
    # Implement the creation
    metadata.create_all() 
    
if not engine.dialect.has_table(engine, dbTradesHistoryTable):  # If table don't exist, Create.
    metadata = MetaData(engine)
    
    # Create a table with the appropriate Columns
    Table(dbTradesHistoryTable, metadata,
          Column('price', Float, nullable=False), 
          Column('buy_sell', String,  nullable=False),  
          Column('market_limit', String),
          Column('miscellaneous', String),
          Column('time', DateTime,  nullable=False),
          Column('volume', Float, nullable=False))
    # Implement the creation
    metadata.create_all()

In [12]:
BalanceTime = 10 
espera = 10 #segundas
Ejecutar = True
i = 0
cantEject = int(3600/espera)*4 # = 3 horas aprox
last = 0
tradesQuery =  {'pair': 'XXBTZUSD'}

BalanceColNames = ['Time','close','ask','bid','balanceRatio','volbuy','volsell','unbalance']
TradesColsNames = ['price', 'volume', 'time', 'buy_sell', 'market_limit', 'miscellaneous']

lapTradesCount = 0
totalTradesCount = 0
lastKnowTradeTime = datetime.now()  - timedelta(minutes=1)

if  engine.dialect.has_table(engine, dbTradesHistoryTable):
    tradesHistory = pd.read_sql(dbTradesHistoryTable, con=engine)
    if len(tradesHistory) > 0:
        lastKnowTradeTime = tradesHistory['time'].max()    

if  engine.dialect.has_table(engine, dbBalanceHistoryTable) & engine.dialect.has_table(engine, dbTradesHistoryTable):
    #while Ejecutar:
    #for i in range(cantEject):
    for i in range(5):
        try:
            
            BalanceHistory =pd.DataFrame(columns=BalanceColNames)
            
            startTime = datetime.now() 

            #i += 1
            #solicitar datos
            kapi = k.API()
            response = kapi.query_public('Trades',tradesQuery)
            #<price>, <volume>, <time>, <buy/sell>, <market/limit>, <miscellaneous>
            # check error y extraer datos
            error = response['error']
            if len(error) == 0:
                trades = pd.DataFrame(response['result']['XXBTZUSD'])
                # formatear datos
                trades.columns = TradesColsNames
                trades['price'] = trades['price'].apply(strToNum)
                trades['time'] = trades['time'].apply(unixtoDate)
                trades['volume'] = trades['volume'].apply(strToNum)
                trades.set_index('price',inplace=True)

                # filtrar ultima periodo
                intervalo = trades['time'].max() - timedelta(minutes=BalanceTime)
                trades = trades[trades['time'] >= intervalo]

                #separar compras y ventas
                buys = trades[trades['buy_sell'] == 'b']
                sells = trades[trades['buy_sell'] == 's']
                buys = buys.sort_index()
                sells = sells.sort_index(ascending=False)

                #agrupar y sumar volumen por precio para compras y ventas
                sbuy = buys['volume']
                sbuy = sbuy.groupby('price').sum()
                ssell = sells['volume']
                ssell = sells.groupby('price').sum()
                ssell = ssell * -1

                #acumular y obtener delta
                mb =pd.concat([sbuy,ssell],axis=1)
                mb.columns = ['buy','sell']
                mb.fillna(0, inplace=True)
                mb['buy'] = mb['buy'].cumsum()
                mb['sell'] = reverseCumSum(mb['sell'])
                mb['delta'] = mb['buy'] + mb['sell']
                mb['c'] = mb['buy']*0

                #graficar Balance
                #mb.plot(figsize=(15,5), kind='area', stacked=False);

                # balance
                volbuy = mb['buy'].sum()
                volsell = mb['sell'].sum()
                unbalance = mb['delta'].sum()
                balanceRatio = volbuy / (volbuy - volsell)
                print()
                print('lap {0} - at {1}:'.format(i , startTime))
                print('balanceRatio: {0}'.format(balanceRatio))
                print('volbuy: {b}-BTCUSD | volsell: {s}-BTCUSD'.format(b=volbuy,s=volsell))
                print('-> Unbalance: {u}-BTCUSD'.format(u=unbalance))

                #obtener precio actual
                kapi = k.API()
                tikerResponce = kapi.query_public('Ticker', {'pair': 'XXBTZUSD'})
                #<pair_name> = pair name
                #    a = ask array(<price>, <whole lot volume>, <lot volume>),
                #    b = bid array(<price>, <whole lot volume>, <lot volume>),
                #    c = last trade closed array(<price>, <lot volume>),
                #    v = volume array(<today>, <last 24 hours>),
                #    p = volume weighted average price array(<today>, <last 24 hours>),
                #    t = number of trades array(<today>, <last 24 hours>),
                #    l = low array(<today>, <last 24 hours>),
                #    h = high array(<today>, <last 24 hours>),
                #    o = today's opening price
                currentTime = datetime.now()
                errorTiker = tikerResponce['error']
                if len(errorTiker) == 0:
                    close = pd.DataFrame(tikerResponce['result']['XXBTZUSD']['c'])[0][0]
                    ask = pd.DataFrame(tikerResponce['result']['XXBTZUSD']['a'])[0][0]
                    bid = pd.DataFrame(tikerResponce['result']['XXBTZUSD']['b'])[0][0]
                    print('close: {0}'.format(close))
                    #guardar historia balance
                    newBalance = [currentTime,strToNum(close),strToNum(ask),strToNum(bid),balanceRatio,volbuy,volsell,unbalance]
                    BalanceHistory.loc[len(BalanceHistory)] = newBalance
                else:
                    print(errorTiker)

                BalanceHistory = BalanceHistory.set_index(pd.DatetimeIndex(BalanceHistory['Time']))
                BalanceHistory.drop('Time', axis=1,inplace=True)
                BalanceHistory.to_sql(dbBalanceHistoryTable,engine, if_exists='append')
                
                # agregar a al historial de trades
                if len(trades[trades['time'] > lastKnowTradeTime]) > 0:
                    trades= trades[trades['time'] > lastKnowTradeTime]
                    lastKnowTradeTime = trades['time'].max()
                    lapTradesCount = len(trades)
                    totalTradesCount = totalTradesCount + lapTradesCount
                    trades.to_sql(dbTradesHistoryTable,engine, if_exists='append')
                    print('{0} new Trades | {1} Total Trades'.format(lapTradesCount, totalTradesCount))
                else:
                    print('no new trades from {0}'.format(lastKnowTradeTime))
            else:
                print(error)
        except:
            print ("Unexpected error: {0}".format(sys.exc_info()[0]))
            raise

        time.sleep(espera)
        lapTime = datetime.now()
        print( 'elapsed {0} sec'.format(PassTime(startTime, lapTime)))
        i = i + 1



lap 0 - at 2017-10-11 23:32:38.055643:
balanceRatio: 0.5114382196525309
volbuy: 257.30336185000004-BTCUSD | volsell: -245.7942792-BTCUSD
-> Unbalance: 11.509082649999998-BTCUSD
close: 4841.00000
26 new Trades | 26 Total Trades
elapsed 12 sec

lap 1 - at 2017-10-11 23:32:50.408349:
balanceRatio: 0.5522896061051776
volbuy: 245.72580191000003-BTCUSD | volsell: -199.19620856-BTCUSD
-> Unbalance: 46.52959335000002-BTCUSD
close: 4844.90000
3 new Trades | 29 Total Trades
elapsed 13 sec

lap 2 - at 2017-10-11 23:33:03.426094:
balanceRatio: 0.5522896061051776
volbuy: 245.72580191000003-BTCUSD | volsell: -199.19620856-BTCUSD
-> Unbalance: 46.52959335000002-BTCUSD
close: 4844.90000
no new trades
elapsed 15 sec

lap 3 - at 2017-10-11 23:33:18.839976:
balanceRatio: 0.5418959053728368
volbuy: 234.36907291000003-BTCUSD | volsell: -198.12925488-BTCUSD
-> Unbalance: 36.23981803000001-BTCUSD
close: 4844.90000
no new trades
elapsed 15 sec

lap 4 - at 2017-10-11 23:33:34.839891:
balanceRatio: 0.541895905

In [24]:
# read from bddbb
BalanceHistory = pd.read_sql(dbBalanceHistoryTable, con=engine)

In [25]:
BalanceHistory = BalanceHistory.set_index(pd.DatetimeIndex(BalanceHistory['Time']))
BalanceHistory.drop('Time', axis=1,inplace=True)

In [26]:
BalanceHistory.tail(20)

,close,ask,bid,balanceRatio,volbuy,volsell,unbalance
Time,,,,,,,
2017-10-11 23:59:02.562272,4855.7,4849.7,4840.6,0.231411,35.042421,-116.386703,-81.344281
2017-10-12 00:00:01.951668,4855.7,4849.1,4842.9,0.231411,35.042421,-116.386703,-81.344281
2017-10-12 00:01:01.245060,4849.0,4848.9,4843.1,0.285430,43.742195,-109.507778,-65.765583
2017-10-12 00:02:00.661458,4848.6,4848.5,4843.4,0.302628,48.973435,-112.853823,-63.880389
2017-10-12 00:02:59.900847,4833.4,4843.5,4843.5,0.116169,61.284955,-466.262787,-404.977832
2017-10-12 00:03:59.149235,4848.2,4848.7,4848.5,0.127842,68.421751,-466.782587,-398.360836
2017-10-12 00:04:57.995601,4848.0,4847.9,4846.1,0.198908,37.056337,-149.242947,-112.186610
2017-10-12 00:05:57.252990,4847.7,4847.7,4846.4,0.178052,34.640889,-159.913534,-125.272645
2017-10-12 00:06:56.572383,4847.7,4847.9,4846.7,0.191948,37.727131,-158.821534,-121.094403


In [27]:
len(BalanceHistory)

36

In [32]:
# read from bddbb
tradesHistory = pd.read_sql(dbTradesHistoryTable, con=engine)

In [33]:
tradesHistory.set_index('price',inplace=True)

In [34]:
tradesHistory.tail()

,buy_sell,market_limit,miscellaneous,time,volume
price,,,,,
4840.2,s,l,,2017-10-12 00:45:25.985600,0.289398
4839.9,s,l,,2017-10-12 00:45:26.000200,0.130396
4839.2,b,l,,2017-10-12 00:45:30.104000,1.580206
4840.5,b,l,,2017-10-12 00:45:30.117900,0.214291
4840.5,b,l,,2017-10-12 00:45:30.122900,0.205503


In [23]:
len(tradesHistory)

9